## Imports, enums, dataclasses, helper curve

### Section 1: Setup and contract definitions

- **Imports**:
  - `dataclasses`, `typing`, `datetime` → for clean data structures and type hints.  
  - `numpy`, `matplotlib`, `scipy.optimize` → typical quant libraries (only `numpy` is actually used here).
  - `Enum` → lets us define a small set of named constants for futures types.

- **`FuturesType` enum**:
  - Labels different interest rate futures: EURODOLLAR, 1M SOFR, 3M SOFR.
  - Makes the code more readable than using raw strings like `"SR3"` everywhere.

- **`FuturesSpecs` dataclass**:
  - Represents *one futures contract*.
  - Fields: `contract_code`, `expiry_date`, `notional`, `tick_size`, `tick_value`.
  - `@dataclass` auto-creates the constructor and other boilerplate for us.
  - Methods:
    - `implied_rate`: turns a **price** (e.g. 95.00) into an **interest rate** (e.g. 5%).
    - `contract_price`: does the reverse (rate → price).
    - `set_price` / `set_rate`: store the internal `_price` / `_rate` used by those properties.

- **`OISCurve`** (simplified):
  - Represents a discount curve (normally used to get discount factors for cash flows).
  - Here it’s just a **flat curve** for teaching: `df(t)` always returns the same value.
  - Stores `today` as the curve’s reference date.


In [7]:
from dataclasses import dataclass
from typing import Dict, List, Optional
from datetime import datetime, timedelta

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from enum import Enum


class FuturesType(Enum):
    EURODOLLAR = "ED"   # 3M LIBOR (legacy)
    SOFR_1M = "SR1"     # 1M SOFR
    SOFR_3M = "SR3"     # 3M SOFR


@dataclass
class FuturesSpecs:
    """Futures contract specifications."""
    contract_code: str
    expiry_date: datetime
    notional: float = 1_000_000.0    # $1M per contract
    tick_size: float = 0.005         # 0.5 bp minimum
    tick_value: float = 12.50        # $12.50 per 0.5 bp

    @property
    def implied_rate(self) -> float:
        """Convert price to implied rate (e.g., 95.00 -> 5.0%)."""
        if hasattr(self, "_price"):
            return (100.0 - self._price) / 100.0
        raise AttributeError("Price not set")

    @property
    def contract_price(self) -> float:
        """Convert rate to price (e.g., 5.0% -> 95.00)."""
        if hasattr(self, "_rate"):
            return 100.0 - (self._rate * 100.0)
        raise AttributeError("Rate not set")

    def set_price(self, price: float) -> None:
        self._price = price

    def set_rate(self, rate: float) -> None:
        self._rate = rate


class OISCurve:
    """
    Very simple OIS discount curve placeholder.

    In the example, it's used as a flat curve: df(t) = 1.0 for all t.
    """

    def __init__(self, dates: List[datetime], dfs: List[float]):
        self.dates = dates
        self.dfs = dfs
        self.today = dates[0]

    def df(self, t: float) -> float:
        # For educational purposes, just return a flat discount factor
        return self.dfs[0]


## IR Future class

### Section 2: Interest rate futures pricing and risk

- **`InterestRateFuture` class**:
  - Ties together:
    - `specs` → the futures contract details (`FuturesSpecs`).
    - `curve` → the OIS discount curve (`OISCurve`).
    - `vol_surface` → a simple volatility surface keyed by expiry (e.g. `{0.25: 0.20}`).

- **Basic conversions**:
  - `price_to_rate` and `rate_to_price`:
    - Convert between quoted futures **price** (e.g. 95.00) and **annualised rate** (e.g. 5%).
    - Formula: `rate = (100 - price) / 100`.

- **Convexity adjustment** (`convexity_adjustment`):
  - Futures prices are biased vs the true forward rate because of daily margining and rate volatility.
  - This function estimates that bias using:
    - `"hull"`: Hull–White-style approximation with volatility and time.
    - `"heston"`: a toy Heston-style formula (vol, vol-of-vol, correlation).
    - `"exact"`: not implemented (would need a full interest rate model).

- **Implied forward rate** (`implied_forward_rate`):
  - Steps:
    1. Compute time to expiry in years.
    2. Get discount factors from the curve at `expiry` and `expiry + tenor`.
    3. Derive the **forward rate** from those discount factors.
    4. Subtract the **convexity adjustment** to get the fair futures rate.

- **Fair futures price** (`fair_price`):
  - Converts the convexity-adjusted forward rate back into a quoted **futures price**.

- **Margin simulation** (`margin_simulation`):
  - Takes a **price path** (array of daily futures prices).
  - For each step:
    - Calculates price change → tick change → P&L.
    - Updates the margin account and records P&L as “margin calls”.
    - If margin drops below 75% of initial margin, it “tops up” back to the initial level.
  - Returns summary stats: final margin, min/max margin move, total negative calls.

- **Hedge ratio** (`hedge_ratio`):
  - Goal: how many futures contracts to hedge a swap with a given **DV01**.
  - Futures DV01 is assumed to be \$25 per bp per contract.
  - Basic hedge: `contracts = swap_dv01 / futures_dv01`.
  - Then it scales this by a **convexity factor** to adjust for model mismatch.


In [8]:
class InterestRateFuture:
    """
    Production-grade interest rate futures with convexity adjustment
    and margin simulation (simplified for educational purposes).
    """

    def __init__(
        self,
        specs: FuturesSpecs,
        discount_curve: OISCurve,
        volatility_surface: Optional[Dict[float, float]] = None,
    ):
        """
        Args:
            specs: Futures contract specifications.
            discount_curve: OIS discount curve for pricing.
            volatility_surface: Vol by expiry (e.g., {0.25: 0.20}).
        """
        self.specs = specs
        self.curve = discount_curve
        self.vol_surface = volatility_surface or {}
        self.margin_account = 0.0
        self.contracts = 0

    @staticmethod
    def price_to_rate(price: float) -> float:
        """Convert quoted price to annualised rate."""
        return (100.0 - price) / 100.0

    @staticmethod
    def rate_to_price(rate: float) -> float:
        """Convert annualised rate to quoted price."""
        return 100.0 - (rate * 100.0)

    def convexity_adjustment(
        self,
        expiry: float,
        tenor: float,
        method: str = "hull",
    ) -> float:
        """
        Calculate convexity adjustment using various models.

        Args:
            expiry: Time to expiry (years).
            tenor: Underlying rate tenor (years).
            method: 'hull', 'heston', or 'exact'.

        Returns:
            Convexity adjustment in decimal (e.g., 0.0005 = 5 bp).
        """
        if method == "hull":
            # Hull-White approximation: ~ 0.5 * σ^2 * T1 * (T1 + T2)
            vol = self.vol_surface.get(expiry, 0.20)  # default 20%
            return 0.5 * (vol ** 2) * expiry * (expiry + tenor)

        elif method == "heston":
            # Simplified Heston-style convexity term
            vol = self.vol_surface.get(expiry, 0.20)
            vol_of_vol = 0.5  # Assumed
            rho = -0.5        # Assumed correlation
            kappa = 1.0       # Mean reversion
            return (rho * vol * vol_of_vol / kappa) * (expiry ** 2)

        else:
            # Exact method would require a full term structure model
            raise NotImplementedError(
                "Exact method requires term structure model"
            )

    def implied_forward_rate(self) -> float:
        """
        Extract forward rate from discount curve and subtract
        convexity bias to get futures fair value.
        """
        # Time to expiry in years
        expiry = (self.specs.expiry_date - self.curve.today).days / 365.25

        # Assume 3M underlying tenor
        tenor = 0.25
        t_end = expiry + tenor

        df_expiry = self.curve.df(expiry)
        df_end = self.curve.df(t_end)

        forward_rate = (df_expiry / df_end - 1.0) / tenor

        # Subtract convexity adjustment
        ca = self.convexity_adjustment(expiry, tenor)
        futures_rate = forward_rate - ca
        return futures_rate

    def fair_price(self) -> float:
        """Calculate fair futures price accounting for convexity."""
        rate = self.implied_forward_rate()
        return self.rate_to_price(rate)

    def margin_simulation(
        self,
        price_path: np.ndarray,
        initial_margin: float = 1500.0,
    ) -> Dict[str, float]:
        """
        Simulate margin account given a price path.

        Args:
            price_path: Array of daily futures prices.
            initial_margin: Initial margin requirement.

        Returns:
            Dict containing margin analytics.
        """
        self.margin_account = initial_margin
        margin_calls: List[float] = []

        for i in range(1, len(price_path)):
            # Price change in ticks (0.01 = 1 bp)
            price_change = price_path[i] - price_path[i - 1]
            tick_change = price_change / 0.01

            # $25 per 1bp per contract (since tick_value = 12.5 per 0.5bp)
            pnl = tick_change * 25.0 * self.contracts
            self.margin_account += pnl
            margin_calls.append(pnl)

            # Check for margin violation (e.g. 75% of initial margin)
            if self.margin_account < initial_margin * 0.75:
                # Variation margin call resets account to initial_margin
                self.margin_account = initial_margin

        return {
            "final_margin": self.margin_account,
            "min_margin": min(margin_calls) if margin_calls else 0.0,
            "max_margin": max(margin_calls) if margin_calls else 0.0,
            "total_calls": sum(c for c in margin_calls if c < 0),
        }

    def hedge_ratio(self, swap_dv01: float) -> int:
        """
        Calculate number of futures contracts to hedge swap DV01.

        Args:
            swap_dv01: DV01 of swap position (in $ per bp).

        Returns:
            Number of contracts (rounded).
        """
        # Futures DV01 = $25 per bp per contract
        futures_dv01 = 25.0

        # Basic hedge ratio
        contracts = int(round(swap_dv01 / futures_dv01))

        # Adjust for convexity mismatch
        expiry = (self.specs.expiry_date - self.curve.today).days / 365.25
        convexity_factor = 1.0 + self.convexity_adjustment(expiry, 0.25) * 100.0

        return int(round(contracts * convexity_factor))


# (Section 3 code here: the if __name__ == "__main__": block)

```
__main__

```



### Section 3: Example – hedging a swap with a SOFR futures strip

- Sets `today` and builds a **flat OIS curve**:
  - `OISCurve([today], [1.0])` → super simplified, just for demo.

- Creates a strip of 4 SOFR futures:
  - Expiries: every 90 days from today (roughly quarterly).
  - For each expiry:
    - Builds `FuturesSpecs` with a contract code (e.g. `"SR3H21"`).
    - Sets an **upward-sloping** price curve (e.g. 95.00, 95.5, 96.0, ...).
    - Creates an `InterestRateFuture` with a simple volatility surface.
    - Sets `contracts = 1` as a placeholder for margin simulation.

- Hedging a 1-year swap:
  - Assumes the swap has DV01 = \$85,000.
  - Splits that DV01 equally across the 4 futures and calls `hedge_ratio()` on each.
  - Sums up the contracts across the strip to get the **total number of contracts**.

- Prints:
  - The swap DV01.
  - The total futures contracts required.
  - A “convexity-adjusted ratio” vs the naive `swap_dv01 / 25` benchmark.


In [9]:
if __name__ == "__main__":
    # Example: SOFR futures strip for swap hedging
    today = datetime(2025, 11, 22)

    # Build curve (flat for simplicity)
    curve = OISCurve([today], [1.0])

    # Create SOFR futures specs
    expiries = [today + timedelta(days=90 * i) for i in range(1, 5)]
    futures: List[InterestRateFuture] = []

    for i, expiry in enumerate(expiries, start=1):
        specs = FuturesSpecs(
            contract_code=f"SR3H2{i}",
            expiry_date=expiry,
            notional=1_000_000.0,
        )
        # Set an upward-sloping price curve
        specs.set_price(95.00 + (i - 1) * 0.5)

        fut = InterestRateFuture(
            specs=specs,
            discount_curve=curve,
            volatility_surface={0.25: 0.18, 0.5: 0.20, 0.75: 0.22, 1.0: 0.24},
        )
        # Example: assume we hold 1 contract each for margin sims
        fut.contracts = 1
        futures.append(fut)

    # Calculate strip hedge for 1-year swap
    swap_dv01 = 85_000  # hypothetical DV01 in $
    total_contracts = sum(f.hedge_ratio(swap_dv01 / 4.0) for f in futures)

    print("\n=== SOFR Futures Strip Hedge ===")
    print(f"Swap DV01: ${swap_dv01:,.0f}")
    print(f"Total Contracts Needed: {total_contracts}")
    print(
        f"Convexity-Adjusted Ratio: "
        f"{total_contracts / (swap_dv01 / 25.0):.2f}"
    )



=== SOFR Futures Strip Hedge ===
Swap DV01: $85,000
Total Contracts Needed: 7543
Convexity-Adjusted Ratio: 2.22
